In [1]:
import re
import requests
import functools
import pandas as pd
from bs4 import BeautifulSoup
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from lxml import etree,html as lhtml

In [2]:
#отключаем скроллер

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
} 

<IPython.core.display.Javascript object>

In [4]:
url = 'https://www.respublica.ru/'

In [5]:
def authors(id_file):
    with open(id_file) as f:
        return [url + 'authors/'+ re.sub(r'\n', '', author) + '?page=1' for author in f]

In [6]:
def get_page(url, attempts=5, sleep_time=0.01, **kwargs):
    
    for _ in range(attempts):
        
        page = requests.get(url,**kwargs)
        if page.ok:
            return page
        
        time.sleep(sleep_time)
        
    print(url, page.status_code)

In [7]:
def author_books(author_url):

    author_page = get_page(author_url).text
    soup = BeautifulSoup(author_page, 'html.parser')

    cut_url = list(map(lambda x: x['href'], soup.find_all('a', class_="rd-listing-product-item__image-wrapper")))
    urls = ['https://www.respublica.ru'+ i for i in cut_url]

    next_page = soup.find('a', class_='pagination-next')
    
    if next_page:
        return urls + author_books(re.sub(r'=\d', '={}'.format(next_page['href'][-1]), author_url))
    else:
        return urls

In [8]:
%%time
authors_url_list = authors('authors.txt')

with ThreadPool(processes=10) as pool:
    book_urls = pool.map(author_books, authors_url_list)
    
all_books = [each_book for books in book_urls for each_book in books] 
# print(all_books)

CPU times: user 11.4 s, sys: 469 ms, total: 11.9 s
Wall time: 22.9 s


In [9]:
def logger(func):
    @functools.wraps(func)
    def wrapper(a):
        result = func(a)
#         if counter is None:
#             counter = 0
#         counter += 1
#         print(counter)
        with open('./parser_logfile', 'a') as f_output:
            print("{}".format(result['Название']), file=f_output)
        return result
    return wrapper

In [10]:
@logger
def process_page(book_url):
    
    book = get_page(book_url).text
    tree = lhtml.fromstring(book)
    
    preview = tree.xpath('//a[@class="download-pdf"]//attribute::href')
    if preview:
        preview = url + preview[0]
    else:
        preview = None
        
    old_price = tree.xpath('//span[@class="prev"]')
    if old_price:
        old_price = re.sub(r'\D', '', old_price[0].text)
    else:
        old_price = None
        
    if tree.xpath('//span[@class="current"]'):
        availible = True
    else:
        availible = False
    
    categories = ':'.join(tree.xpath('//span[@class="rd-page-breadcrumbs-item"]//span[@itemprop="name"]//text()'))
    
    section_one = tree.xpath('//div[@class="rd-rating-stars"]//attribute::content')    
    if section_one:
        review_number = section_one[0]
        ratings_number = section_one[1]
        rating = section_one[3]
    else:
        review_number = None
        ratings_number = None
        rating = None
    
    section_two = tree.xpath('//div[@class="rd-page-product__desc-params"]')[0]
    name = section_two.xpath('.//p/*[contains(@itemprop, "name")]/text()')
    value = section_two.xpath('.//p/*[contains(@itemprop, "value")]/text()')
    
    info = {
        'Id' : tree.xpath('//span[@itemprop="sku"]')[0].text,
        'Url' : book_url,
        'Название' : tree.xpath('//h1[@class="rd-page-product__title"]')[0].text,
        'Автор' : ';'.join(tree.xpath('//a[@itemprop="brand"]//text()')),
        'Превью' : preview,
        'Изображение' : url + tree.xpath('//img[@class="rd-page-product__img"]//attribute::src')[0],
        'Описание' : re.sub(r'\s+', ' ', \
                            '\n'.join(tree.xpath('//div[@class="rd-page-product__desc-body"]//text()'))),
        'Цена' : re.sub(r'\D','', tree.xpath('//span[@class="num"]')[0].text),
        'Цена (старая)' : old_price,
        'В наличии' : availible,
        'Категория' : categories,
        'Число отзывов' : review_number,
        'Число оценок' : ratings_number,
        'Оценка' : rating
    }
    
    info.update(zip(name, value))
    return info

In [11]:
process_page('https://www.respublica.ru/knigi/hudozhestvennaya-literatura/fantastika-i-fentezi/511072-geyman-n-istoriya-s-kladbischem-mastera-magicheskogo-realizma-t-o')

{'Id': '511072',
 'Url': 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/fantastika-i-fentezi/511072-geyman-n-istoriya-s-kladbischem-mastera-magicheskogo-realizma-t-o',
 'Название': 'История с кладбищем',
 'Автор': 'Нил Гейман',
 'Превью': 'https://www.respublica.ru//items/370917/download_preview_pdf',
 'Изображение': 'https://www.respublica.ru//uploads/00/00/00/br/xg/large_64fbf94891a4f13a.jpg',
 'Описание': 'В ночь, когда погибла вся его семья, крошечный мальчик каким-то чудом оказывается на улице и даже добирается до старого кладбища, где находит приют на долгие годы. Обитатели кладбища, призраки, вампир и оборотень, дают мальчику имя, воспитывают и опекают его. На кладбище — и в большом, человеческом мире — его ждет множество приключений, которые помогут ему повзрослеть и полюбить мир живых, опасный, волнующий и манящий.',
 'Цена': '350',
 'Цена (старая)': None,
 'В наличии': True,
 'Категория': ' Книги: Художественная литература: Фантастика и фэнтези',
 'Число отзывов

In [12]:
process_page('https://www.respublica.ru/knigi/hudozhestvennaya-literatura/fantastika-i-fentezi/517325-strugatskiy-a-strugatskiy-b-sobranie-sochineniy-1973-1978-strugatskie-sobranie-sochineniy-t-o')

{'Id': '517325',
 'Url': 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/fantastika-i-fentezi/517325-strugatskiy-a-strugatskiy-b-sobranie-sochineniy-1973-1978-strugatskie-sobranie-sochineniy-t-o',
 'Название': 'Собрание сочинений 1973-1978',
 'Автор': 'Аркадий и Борис Стругацкие',
 'Превью': 'https://www.respublica.ru//items/377275/download_preview_pdf',
 'Изображение': 'https://www.respublica.ru//uploads/00/00/00/c7/bw/large_83c6f808d00aa335.jpg',
 'Описание': ' Аркадий (1925-1991) и Борис (1933-2012) Стругацкие - русские советские писатели-фантасты, поднявшие отечественную фантастику до высот мирового уровня. Переведенные на все основные языки, изданные суммарным тиражом более 500 миллионов экземпляров, их книги до сих пор экранизируются, активно обсуждаются и служат источником вдохновения для нового поколения писателей и читателей. В этот том вошли "За миллиард лет до конца света"; "Град обреченный"; "Повесть о дружбе и недружбе". ',
 'Цена': '635',
 'Цена (старая)': No

In [13]:
with ThreadPool(processes=10) as pool:
    result = pool.map(process_page, all_books)
    
pool.join()

df = pd.DataFrame(result)
df.sort_values(by=['Id'], inplace=True)
df = df.fillna('')

with open('result.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [15]:
pd.set_option('display.max_columns', 100)

error_number = df.shape[0] - len(all_books)
if error_number == 0:
    print('Количество ошибок = {}. Все книги успешно обкачаны'.format(error_number))
    
df.reset_index(inplace=True, drop=True)
df.head(10)

Количество ошибок = 0. Все книги успешно обкачаны


,Id,Url,Название,Автор,Превью,Изображение,Описание,Цена,Цена (старая),В наличии,Категория,Число отзывов,Число оценок,Оценка,ISBN,Издательство,Серия,Обложка,Формат,Количество страниц,Год издания,Язык,Раздел,Направление,Страна-производитель,"Вес, г",Возрастные ограничения,Жанр,Герои,Эпоха,Возраст,Иллюстратор,Тип,Вид бумаги,Материал,"Размер, см",Пол,Назначение,Тематика,Упаковка,Рисунок,Иллюстрации
0,000033,https://www.respublica.ru/knigi/hudozhestvenna...,Много шума из ничего,Уильям Шекспир,,https://www.respublica.ru//uploads/00/00/00/cq...,В настоящем издании под одной обложкой объедин...,100,140,False,Книги: Художественная литература: Поэзия,,,,978-5-389-03203-3,Азбука,,Мягкая,11 х 18,224,2015,Русский,Зарубежные,,Россия,,,,,,,,,,,,,,,,,
1,000056,https://www.respublica.ru/knigi/hudozhestvenna...,Двенадцатая ночь,Уильям Шекспир,,https://www.respublica.ru//uploads/01/00/00/24...,Перу великого английского драматурга Уильяма Ш...,100,140,False,Книги: Художественная литература: Поэзия,,,,978-5-389-03202-6,Азбука,,Мягкая,11 х 18,192,2011,Русский,Зарубежные,,Россия,,,,,,,,,,,,,,,,,
2,021678,https://www.respublica.ru/knigi/hudozhestvenna...,N or M?,Agatha Christie,,https://www.respublica.ru//uploads/00/00/00/68...,Tommy & Tuppence are hired to track down warti...,610,,False,Книги: Художественная литература: Книги на ин...,,,,,HarperCollins,,Мягкая,10 х 17,304,2001,Английский,Зарубежные,,США,,,Детектив,,,,,,,,,,,,,,
3,021679,https://www.respublica.ru/knigi/hudozhestvenna...,The Secret Adversary,Agatha Christie,,https://www.respublica.ru//uploads/00/00/00/68...,Agatha Christie's first Tommy and Tuppence mys...,610,,False,Книги: Художественная литература: Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,400,2011,Английский,Зарубежные,,США,,,Детектив,,,,,,,,,,,,,,
4,021680,https://www.respublica.ru/knigi/hudozhestvenna...,By the Pricking of My Thumbs,Agatha Christie,,https://www.respublica.ru//uploads/01/00/00/68...,An old woman in a nursing home speaks of a chi...,510,610,False,Книги: Художественная литература: Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,352,2011,Английский,Зарубежные,,США,,,Детектив,,,,,,,,,,,,,,
5,021681,https://www.respublica.ru/knigi/hudozhestvenna...,Partners in Crime,Agatha Christie,,https://www.respublica.ru//uploads/00/00/00/68...,Agatha Christie's complete Tommy and Tuppence ...,610,,False,Книги: Художественная литература: Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,352,2001,Английский,Зарубежные,,США,,,Детектив,,,,,,,,,,,,,,
6,021700,https://www.respublica.ru/knigi/hudozhestvenna...,Evil Under the Sun,Agatha Christie,,https://www.respublica.ru//uploads/00/00/00/0p...,The beautiful bronzed body of Arlena Stuart la...,540,,False,Книги: Художественная литература: Книги на ин...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,021702,https://www.respublica.ru/knigi/hudozhestvenna...,The Murder on the Links,Agatha Christie,,https://www.respublica.ru//uploads/01/00/00/68...,"On a French golf course, a millionaire is foun...",639,,False,Книги: Художественная литература: Книги на ин...,,,,,HarperCollins,,Мягкая,13 x 20,288,2017,,,,США,,,,,,,,,,,,,,,,,
8,021703,https://www.respublica.ru/knigi/hudozhestvenna...,The ABC Murders,Agatha Christie,,https://www.respublica.ru//uploads/01/00/00/5d...,"There's a serial killer on the loose, working ...",610,,False,Книги: Художественная литература: Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,278,2010,Английский,Зарубежные,,США,,,Детектив,,,,,,,,,,,,,,
9,021706,https://www.respublica.ru/knigi/hudozhestvenna...,Death on the Nile,Agatha Christie,,https://www.respublica.ru//uploads/01/00/00/60...,The tranquillity of a cruise along the Nile wa...,490,,False,Книги: Художественная литература: Книги на ин...,,,,978-0-00-711932-5,HarperCollins,Poirot,Мягкая,11 х 17,416,2001,Английский,Зарубежные,,США,,,Детектив,,,,,,,,,,,,,,
